<a href="https://colab.research.google.com/github/morgan03153/game/blob/master/morgan_stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas_datareader

In [ ]:
!pip install yfinance pandas numpy scikit-learn keras google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client google-cloud-storage


畫曲線圖

In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pandas_datareader.data as web

# 設定時間範圍
end_date = datetime.today()
start_date = end_date - timedelta(days=60)

# 獲取1個月期美國國債殖利率
treasury_1mo = web.DataReader('DGS1MO', 'fred', start_date, end_date)

# 獲取其他數據
vix_data = yf.download('^VIX', start=start_date, end=end_date)
voo_data = yf.download('VOO', start=start_date, end=end_date)
nvda_data = yf.download('NVDA', start=start_date, end=end_date)
msft_data = yf.download('MSFT', start=start_date, end=end_date)
avgo_data = yf.download('AVGO', start=start_date, end=end_date)

# 合併數據以便於繪圖
combined_data = pd.DataFrame({
    '1-Month Treasury Yield': treasury_1mo['DGS1MO'],
    'VIX': vix_data['Close'],
    'VOO': voo_data['Close'],
    'NVDA': nvda_data['Close'],
    'MSFT': msft_data['Close'],
    'AVGO': avgo_data['Close']
})

# 正規化處理，使每條曲線的第一點為100
normalized_data = combined_data.apply(lambda x: (x / x.iloc[0]) * 100)

# 定義不同的marker
markers = ['o', 'x', 's', 'D', '^', 'v']

# 繪製所有曲線在同一張圖上，並在每一點加上不同的標記
plt.figure(figsize=(14, 8))

for i, column in enumerate(normalized_data.columns):
    plt.plot(normalized_data.index, normalized_data[column], label=column, marker=markers[i])

plt.title('Normalized 1-Month Treasury Yield, VIX, and Selected Stocks (Base 100)')
plt.xlabel('Date')
plt.ylabel('Normalized Value (Base 100)')
plt.legend(loc='upper left')
plt.grid(True)

# 调整X轴日期标签为直式显示
plt.xticks(rotation=90)

plt.show()


使用LSTM訓練去預估NVDA在接下來14天內是否曾累積跌10%以上

In [7]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.models import save_model
import joblib
from google.colab import drive
import pandas_datareader.data as web

# 连接Google Drive
drive.mount('/content/drive')

# 設定保存模型和標準化器的路徑
save_path = '/content/drive/My Drive/nvda_lstm/'

# 确保保存路径存在
import os
os.makedirs(save_path, exist_ok=True)

# 設定時間範圍
end_date = datetime.today() - timedelta(days=0)
start_date = end_date - timedelta(days=60)

# 獲取各個股票和指數的數據
nvda_data = yf.download('NVDA', start=start_date, end=end_date)
vix_data = yf.download('^VIX', start=start_date, end=end_date)
voo_data = yf.download('VOO', start=start_date, end=end_date)
msft_data = yf.download('MSFT', start=start_date, end=end_date)
avgo_data = yf.download('AVGO', start=start_date, end=end_date)

# 獲取美國1個月期國債殖利率數據
# treasury_1mo = web.DataReader('DGS1MO', 'fred', start_date, end_date)


# 填充缺失值
nvda_data = nvda_data.fillna(method='ffill').fillna(method='bfill')
vix_data = vix_data.fillna(method='ffill').fillna(method='bfill')
voo_data = voo_data.fillna(method='ffill').fillna(method='bfill')
msft_data = msft_data.fillna(method='ffill').fillna(method='bfill')
avgo_data = avgo_data.fillna(method='ffill').fillna(method='bfill')
# treasury_1mo = treasury_1mo.fillna(method='ffill').fillna(method='bfill')

#breakpoint()
# 对齐所有数据的索引
combined_data = pd.concat([nvda_data['Close'], vix_data['Close'], voo_data['Close'], msft_data['Close'], avgo_data['Close']], axis=1).dropna()
combined_data.columns = ['NVDA', 'VIX', 'VOO', 'MSFT', 'AVGO']

# 将 NVDA 数据与特征数据对齐
nvda_data_aligned = nvda_data.loc[combined_data.index]

# 标准化数据
scalers = {}
for column in combined_data.columns:
    scaler = MinMaxScaler(feature_range=(0, 1))
    combined_data[column] = scaler.fit_transform(combined_data[column].values.reshape(-1, 1))
    scalers[column] = scaler

nvda_close = nvda_data_aligned['Close'].values.reshape(-1, 1)
nvda_scaler = MinMaxScaler(feature_range=(0, 1))
nvda_scaled = nvda_scaler.fit_transform(nvda_close)

# 創建數據集，y 值使用 NVDA 的股價，判断未来14天内是否累计下跌超过10%
def create_dataset(X, y, look_back=60, forward_days=14):
    Xs, ys = [], []
    for i in range(len(X) - look_back - forward_days):
        Xs.append(X[i:i + look_back])
        # 判断未来14天内是否有累计下跌超过10%的情况
        future_window = y[i + look_back : i + look_back + forward_days]
        max_drop = np.max((y[i + look_back] - future_window) / y[i + look_back])
        ys.append(1 if max_drop >= 0.1 else 0)
    return np.array(Xs), np.array(ys)

look_back = 60
forward_days = 14
X, y_binary = create_dataset(combined_data.values, nvda_scaled, look_back, forward_days)

# 构建LSTM模型
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(25))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 拆分数据集为训练集和测试集
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y_binary[:split], y_binary[split:]

# 训练模型
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_test, y_test))

# 保存模型和標準化器
model_path = os.path.join(save_path, 'nvda_lstm_model_14days.h5')
scaler_paths = {k: os.path.join(save_path, f"{k}_scaler.pkl") for k in scalers}

save_model(model, model_path)

# 保存 scalers
joblib.dump(nvda_scaler, os.path.join(save_path, 'nvda_scaler.pkl'))
for k, scaler in scalers.items():
    joblib.dump(scaler, scaler_paths[k])

print(f"模型已保存到 {model_path}")
for k, path in scaler_paths.items():
    print(f"{k} 的 scaler 已保存到 {path}")


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
<ipython-input-7-7b11773a3ee7>:39: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  nvda_data = nvda_data.fillna(method='ffill').fillna(method='bfill')
<ipython-input-7-7b11773a3ee7>:40: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  vix_data = vix_data.fillna(method='ffill').fillna(method='bfill')
<ipython-input-7-7b11773a3ee7>:41: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  voo_data = voo_data.fillna(method='ffill').fillna(method='bfill')
<ipython-input-7-7b11773a3ee7>:42: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffi

--Return--
None
> <ipython-input-7-7b11773a3ee7>(54)<cell line: 54>()
     52 nvda_data_aligned = nvda_data.loc[combined_data.index]
     53 
---> 54 breakpoint()
     55 # 标准化数据
     56 scalers = {}

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user


IndexError: tuple index out of range

預測程式

In [ ]:
from keras.models import load_model
import numpy as np
import pandas as pd
import joblib
from datetime import datetime, timedelta
import yfinance as yf
import pandas_datareader.data as web
from google.colab import drive

# 连接Google Drive
drive.mount('/content/drive')

# 設定保存模型和標準化器的路徑
save_path = '/content/drive/My Drive/nvda_lstm/'

# 加载模型和scalers
model_path = os.path.join(save_path, 'nvda_lstm_model_14days.h5')
nvda_scaler = joblib.load(os.path.join(save_path, 'nvda_scaler.pkl'))
scaler_paths = {
    "1-Month Treasury Yield": os.path.join(save_path, '1-Month Treasury Yield_scaler.pkl'),
    "VIX": os.path.join(save_path, 'VIX_scaler.pkl'),
    "VOO": os.path.join(save_path, 'VOO_scaler.pkl'),
    "MSFT": os.path.join(save_path, 'MSFT_scaler.pkl'),
    "AVGO": os.path.join(save_path, 'AVGO_scaler.pkl')
}

model = load_model(model_path)
scalers = {k: joblib.load(path) for k, path in scaler_paths.items()}

# 設定時間範圍
end_date = datetime.today() - timedelta(days=25)
start_date = end_date - timedelta(days=600)

# 獲取各個股票和指數的數據
nvda_data = yf.download('NVDA', start=start_date, end=end_date)
vix_data = yf.download('^VIX', start=start_date, end=end_date)
voo_data = yf.download('VOO', start=start_date, end=end_date)
msft_data = yf.download('MSFT', start=start_date, end=end_date)
avgo_data = yf.download('AVGO', start=start_date, end=end_date)

# 獲取美國1個月期國債殖利率數據
treasury_1mo = web.DataReader('DGS1MO', 'fred', start_date, end_date)

# 填充缺失值
nvda_data = nvda_data.fillna(method='ffill').fillna(method='bfill')
vix_data = vix_data.fillna(method='ffill').fillna(method='bfill')
voo_data = voo_data.fillna(method='ffill').fillna(method='bfill')
msft_data = msft_data.fillna(method='ffill').fillna(method='bfill')
avgo_data = avgo_data.fillna(method='ffill').fillna(method='bfill')
treasury_1mo = treasury_1mo.fillna(method='ffill').fillna(method='bfill')

# 对齐所有数据的索引
combined_data = pd.concat([treasury_1mo, vix_data['Close'], voo_data['Close'], msft_data['Close'], avgo_data['Close']], axis=1).dropna()
combined_data.columns = ['1-Month Treasury Yield', 'VIX', 'VOO', 'MSFT', 'AVGO']

# 将 NVDA 数据与特征数据对齐
nvda_data_aligned = nvda_data.loc[combined_data.index]

# 标准化数据
for column in combined_data.columns:
    combined_data[column] = scalers[column].transform(combined_data[column].values.reshape(-1, 1))

nvda_close = nvda_data_aligned['Close'].values.reshape(-1, 1)
nvda_scaled = nvda_scaler.transform(nvda_close)

# 获取最近60天的数据
look_back = 60
last_lookback = np.array([combined_data.values[-look_back:]])

# 预测未来14天内NVDA股价是否累计下跌超过10%
predicted_value = model.predict(last_lookback)

if predicted_value > 0.5:
    print("预测未来14天内NVDA股价有超过10%的累计下跌风险。")
else:
    print("预测未来14天内NVDA股价没有超过10%的累计下跌风险。")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
<ipython-input-40-eb29eac7b921>:45: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  nvda_data = nvda_data.fillna(method='ffill').fillna(method='bfill')
<ipython-input-40-eb29eac7b921>:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  vix_data = vix_data.fillna(method='ffill').fillna(method='bfill')
<ipython-input-40-eb29eac7b921>:47: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
预测未来14天内NVDA股价没有超过10%的累计下跌风险。
